Pontus Hultkrantz

# [December 2021 : Robot Archery](https://www.janestreet.com/puzzles/current-puzzle/)

<p>After a grueling year filled with a wide variety of <a href="https://www.janestreet.com/puzzles/robot-weightlifting-index/" title="tug of war">robot</a> <a href="https://www.janestreet.com/puzzles/robot-tug-of-war-index/" title="swimming">sporting</a> <a href="https://www.janestreet.com/puzzles/robot-swimming-trials-index/" title="weightlifting">events</a>, we have arrived at the final event of the year: <strong>Robot Archery</strong>. Four robots have qualified for this year’s finals, and have been seeded in the following order:</p>

<table>
  <thead>
    <tr>
      <th>Robot</th>
      <th>Seed </th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Aaron</td>
      <td>1</td>
    </tr>
    <tr>
      <td>Barron  </td>
      <td>2</td>
    </tr>
    <tr>
      <td>Caren</td>
      <td>3</td>
    </tr>
    <tr>
      <td>Darrin </td>
      <td>4</td>
    </tr>
  </tbody>
</table>

<p><br />
The robots will take turns shooting arrows at a target<sup>1</sup>, starting with Aaron and proceeding in order by seed. When it is a given robot’s turn, they shoot a single arrow. If it is closer to the center of the target than <em>all</em> previous arrows by all players, that robot remains in the tournament, going to the back of the queue to await their next turn. Otherwise that robot is eliminated immediately. The last robot remaining in the queue is the winner.</p>

<p>For example, here is how <em>last year’s</em> finals went, in which <strong>Caren</strong> was the winner. (Oddly enough it involved the same robots in the same seeding.)</p>

<table>
  <thead>
    <tr>
      <th>Turn  </th>
      <th>Robot</th>
      <th>Distance</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>Aaron</td>
      <td>10nm</td>
    </tr>
    <tr>
      <td>2</td>
      <td>Barron  </td>
      <td>8nm</td>
    </tr>
    <tr>
      <td>3</td>
      <td>Caren</td>
      <td>7nm</td>
    </tr>
    <tr>
      <td>4</td>
      <td>Darrin  </td>
      <td>1km</td>
    </tr>
    <tr>
      <td>5</td>
      <td>Aaron</td>
      <td>9nm</td>
    </tr>
    <tr>
      <td>6</td>
      <td>Barron</td>
      <td>2nm</td>
    </tr>
    <tr>
      <td>7</td>
      <td>Caren</td>
      <td>1nm</td>
    </tr>
    <tr>
      <td>8</td>
      <td>Barron</td>
      <td>1Ym<sup>2</sup></td>
    </tr>
  </tbody>
</table>

<p><br />
To ten decimal places, what is the probability that <strong>Darrin</strong> will be this year’s winner?
(Or, if you want to send in the <em>exact answer</em>, that’s fine too!)</p>

<p><br /></p>

<p>[1] Each robot is equally skilled. Which is to say: for any region <em>R</em> on the target with nonzero area, the robots all have the same positive probability of landing an arrow within <em>R</em> on any given shot.</p>

<p>[2] It’s a large target.</p>


In [2]:
import numpy as np
from scipy import stats
from queue import SimpleQueue

In [3]:
players = ['Aron', 'Barron', 'Caren', 'Darrin']
nplay = 2
dist = stats.uniform()
#dist = stats.norm()

nsim = 100000
data = np.empty(nsim, dtype=int)
for isim in range(nsim):
    q = SimpleQueue()
    for i in range(nplay):
        q.put(players[i])
    curr_best = 1e6
    while True:
        if q.qsize() == 1:
            winner = q.get(False)
            #print(f"winner is {winner}")
            data[isim] = players.index(winner)
            break
        # Player's turn
        curr_player = q.get(False)
        # Player's result
        res = dist.rvs()
        # Check if best so far
        if abs(res) < curr_best:
            curr_best = abs(res)
            q.put(curr_player)

#data = np.bincount(data)
#data = data / data.sum()
#data
import pandas as pd
data = pd.Series(data).value_counts().reindex(range(nplay)).fillna(0)
data = data/data.sum()
data

0    0.63172
1    0.36828
dtype: float64

In [59]:
import numba

@numba.njit(parallel=False)
def run_sim(nsim, n):
    data = np.zeros((nsim, 2))
    for i in numba.prange(nsim):
        m = 0.5
        for k in range(n):
            rnd = np.random.rand()
            if rnd < m:
                m = rnd
                data[i,0] += 1
        data[i,1] = m
    return data
    
data = run_sim(n = 2, nsim = 25_000_000)
    
df = pd.DataFrame(data, columns=['alive', 'best'])
df = df.groupby('alive').mean()
df

,best
alive,
0.0,0.500000
1.0,0.233417
2.0,0.166765


In [60]:
(df.index*df.best).sum()

0.5669474535814583

## Methodology

Consider $n$ players playing $k$ races.

If all $n$ players are playing the same discrete strategy, due to symmetry they all have the same probability of making the finals, that is $\frac{n}{3n} = \tfrac{1}{3}$.

Now assume the other $n-1$ players are playing the discrete strategy, then the only chance for you to win is
1. You allocate all to one race.
    1. More than you allocated to this race, and ties are broken randomly.
    2. Only you allocted to this race. You win.
2. You allocate to several races.
    1. Any race you allocated to that someone else participates in, you lose since you allocate less than 1.0.
    2. Only you allocated to this race. You win. 

Obviously, 1. is the discrete strategy which has been solved. 2. you only win by being the only player allocating to a race. Hence, you will maximise winning chances by participating with any non-zero value in each race.

Therefore, since you participate in all races, you will make the finals if any of the $k$ races have no other players.

The general problem with $n$ **competitors** and $k$ races can be rephrased as:
"Given $n$ balls and $k$ urns, if all balls are places into urns randomly, what is the probability that at least one urn is empty".

Let $A_i$ be the event that urn $i$ remains empty. It is clear that $\mathbb{P}(A_i) = \left(\frac{k-1}{k}\right)^n$, i.e. all balls must be put in the (k-1) urns. Similarly, the probability that any combination of $j$ urns stay empty is $f(j; k, n) := \left(\frac{k-j}{k}\right)^n$.

Using the [inclusion-exclusion formula ](https://en.wikipedia.org/wiki/Inclusion%E2%80%93exclusion_principle#In_probability)

k = 2:

\begin{align}
\mathbb{P}(A_1 \cup A_2) &= \mathbb{P}(A_1) + \mathbb{P}(A_2) - \mathbb{P}(A_1 \cap A_2) \\
&= 2 f(1; 2, n) - f(2;2,n).
\end{align}

k = 3:
\begin{align}
\mathbb{P}(A_1\cup A_2\cup A_3)&=\mathbb{P}(A_1)+\mathbb{P}(A_2)+\mathbb{P}(A_3)-\mathbb{P}(A_1\cap A_2)-\mathbb{P}(A_1\cap A_3)-\mathbb{P}(A_2\cap A_3)+\mathbb{P}(A_1\cap A_2\cap A_3) \\
&= 3 f(1;3,n) - 3f(2;3,n) + f(3;3,n).
\end{align}




For general $k$ 
\begin{align}
\mathbb{P}\left(\bigcup_{i=1}^k A_i\right) = \sum_{s=1}^k (-1)^{s-1} {k \choose s} f(s;k,n)
\end{align}


where $A_I = \bigcap\limits_{i\in I} A_{i}$ denotes the intersection of all $A_i$ with index in $I$, where $I \subseteq \{1, 2, ..., k \}, |I|=j$.


$, that is $Pr(A_I) = \left(\frac{k-j}{k}\right)^n$.







[inclusion-exclusion](https://math.stackexchange.com/questions/174674/if-n-balls-are-thrown-into-k-bins-what-is-the-probability-that-every-bin-gets-a)
...

## Simulation

## Anlytical solution